# Cancelamento de Clientes - Telco (dataset criado pela IBM para demonstração da ferramenta IBM Cognos Analytics)

### Contém informações sobre uma empresa fictícia de telecomunicações que forneceu serviços de telefonia residencial e internet para 7043 clientes na Califórnia no 3º trimestre.

### Etapa do pipeline - Realizado por Sabrina Otoni da Silva - 2024/01

### Objetivo: Testar combinações dos tratamentos desenvolvidos e modelos de Machine Learning com fine-tuning (o refinamento das predições será realizado no próximo passo após escolha do modelo). 

In [51]:
from pathlib import Path

import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

import sys
import os

automations_dir = os.path.join(os.getcwd(), '../automations')

if automations_dir not in sys.path:
    sys.path.append(automations_dir)

from data_processing import LogTransformer, BoxCoxTransformer, RBFTransformer, KMeansCluster, DropColumns, ServiceTransformer, CategoricalEncoder

import warnings
warnings.filterwarnings('ignore')

In [2]:
datapath = Path('../data')
csv_path = Path(f'{datapath}/d02_intermediate')
preprocesspath = Path('../preprocessing')

In [3]:
X_train = pd.read_csv(f'{csv_path}/X_train.csv')
y_train = pd.read_csv(f'{csv_path}/y_train.csv')

In [4]:
class PassthroughTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X

In [5]:
def get_kmeans():
    return KMeansCluster(model_path='../preprocessing/kmeans_model.pkl', columns_cluster=['Latitude', 'Longitude'])

In [6]:
class ConditionalServiceTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, encoder_type):
        self.service_transformer = ServiceTransformer(columns=['Multiple Lines', 'Online Security', 'Online Backup', 'Device Protection', 
                                                            'Tech Support', 'Streaming TV', 'Streaming Movies'])
        self.encoder_type = encoder_type

    def fit(self, X, y=None):
        if self.encoder_type == 'onehot':
            self.service_transformer.fit(X, y)
            return self
        else:
            return self

    def transform(self, X):
        if self.encoder_type == 'onehot':
            return self.service_transformer.transform(X)
        else:
            return X

In [7]:
def get_transformer(transformer_type: str, columns: list = None):
    if transformer_type == 'log':
        return LogTransformer(model_path='../preprocessing/log_transformer_model.pkl', columns=['Total Charges'])
    elif transformer_type == 'boxcox':
        return BoxCoxTransformer(model_path='../preprocessing/boxcox_transformer_model.pkl', columns=['Total Charges'])
    else:
        return PassthroughTransformer()

In [8]:
def get_rbf():
    return RBFTransformer(model_path='../preprocessing/rbf_transformer_model.pkl', column='Tenure Months') 

In [9]:
def build_pipeline(transformer, cond_encoder_type: str, cat_encoder_type: str, scaler, model):
    pipeline_steps = [
    ('kmeans_cluster', get_kmeans()),
    ('import_drop', DropColumns(drop_columns=["City", "Latitude", "Longitude", "ID"])),
    ('service_transformer', ConditionalServiceTransformer(encoder_type=cond_encoder_type)),
    ('categorical_encoder', CategoricalEncoder(encoder_type=cat_encoder_type, specified_columns=["Gender", "Senior Citizen", "Partner", "Dependents", "Phone Service", "Multiple Lines", "Internet Service",
                                                                  "Online Security", "Online Backup", "Device Protection", "Tech Support", "Streaming TV", "Streaming Movies",
                                                                  "Contract", "Paperless Billing", "Payment Method", "Cluster"])),
    ('transformation', transformer),        
    ('rbf', get_rbf()),                                                              
    ('scaler', scaler),
    ('model', model)
    ]
    return Pipeline(pipeline_steps)

In [10]:
scaler_dict = {
    'standard': StandardScaler(),
    'minmax': MinMaxScaler(),
    'robust': RobustScaler()
    }

In [11]:
model_dict = {
    'dummy': DummyClassifier(strategy='most_frequent'),
    'logistic_regression': LogisticRegression(),
    'svr': SVR(),
    'knn': KNeighborsClassifier(),
    'random_forest': RandomForestClassifier(),
    'xgboost': XGBClassifier(objective='binary:logistic')
    }

In [12]:
pipeline = build_pipeline(
    cond_encoder_type='label',
    cat_encoder_type='label',
    transformer=get_transformer('log'),
    scaler=scaler_dict['minmax'],
    model=model_dict['dummy']
)

In [13]:
param_grid = [
    {
        'kmeans_cluster__active': [True],
        'service_transformer__encoder_type': ['onehot', 'label'], 
        'categorical_encoder__encoder_type': ['onehot', 'label'], 
        'transformation': [get_transformer('log'), get_transformer('boxcox')],
        'rbf__active': [True, False],
        'scaler': [scaler_dict['standard'], scaler_dict['minmax'], scaler_dict['robust']],
        
        'model': [model_dict['dummy']]
    },
    {
        'kmeans_cluster__active': [True],
        'service_transformer__encoder_type': ['onehot', 'label'], 
        'categorical_encoder__encoder_type': ['onehot', 'label'], 
        'transformation': [get_transformer('log'), get_transformer('boxcox')],
        'rbf__active': [True, False],
        'scaler': [scaler_dict['standard'], scaler_dict['minmax'], scaler_dict['robust']],

        'model': [model_dict['logistic_regression']],
        'model__C': [0.01, 0.1, 1.0],
        'model__fit_intercept': [True, False],
        'model__class_weight': ['balanced', None]
    },
    # {
    #     'kmeans_cluster__active': [True],
    #     'service_transformer__encoder_type': ['onehot', 'label'], 
    #     'categorical_encoder__encoder_type': ['onehot', 'label'], 
    #     'transformation': [get_transformer('log'), get_transformer('boxcox')],
    #     'rbf__active': [True, False],
    #     'scaler': [scaler_dict['standard'], scaler_dict['minmax'], scaler_dict['robust']],

    #     'model': [model_dict['svr']],
    #     'model__kernel': ['rbf', 'sigmoid'],
    #     'model__C': [0.01, 0.1, 1.0]
    # },
    # {
    #     'kmeans_cluster__active': [True],
    #     'service_transformer__encoder_type': ['onehot', 'label'], 
    #     'categorical_encoder__encoder_type': ['onehot', 'label'], 
    #     'transformation': [get_transformer('log'), get_transformer('boxcox')],
    #     'rbf__active': [True, False],
    #     'scaler': [scaler_dict['standard'], scaler_dict['minmax'], scaler_dict['robust']],

    #     'model': [model_dict['knn']],
    #     'model__weights': ['uniform', 'distance'],
    #     'model__p': [1, 2]
    # },
    # {
    #     'kmeans_cluster__active': [True],
    #     'service_transformer__encoder_type': ['onehot', 'label'], 
    #     'categorical_encoder__encoder_type': ['onehot', 'label'], 
    #     'transformation': [get_transformer('log'), get_transformer('boxcox')],
    #     'rbf__active': [True, False],
    #     'scaler': [scaler_dict['standard'], scaler_dict['minmax'], scaler_dict['robust']],

    #     'model': [model_dict['random_forest']],
    #     'model__n_estimators': [100, 200],
    #     'model__max_depth': [5, 10, 20, None],
    #     'model__max_features': ['sqrt', 'log2', None],
    #     'model__min_samples_leaf': [1, 50, 100, 200],
    #     'model__class_weight': ['balanced', None]
    # },
    {
        'kmeans_cluster__active': [True],
        'service_transformer__encoder_type': ['onehot', 'label'], 
        'categorical_encoder__encoder_type': ['onehot', 'label'], 
        'transformation': [get_transformer('log'), get_transformer('boxcox')],
        'rbf__active': [True, False],
        'scaler': [scaler_dict['standard'], scaler_dict['minmax'], scaler_dict['robust']],

        'model': [model_dict['xgboost']],
        'model__booster': ['gbtree'],
        'model__learning_rate': [0.01, 0.1],
        'model__gamma': [0.01, 0.1],
        'model__max_depth': [3, 6, 9],
        'model__subsample': [0.6, 0.8],
        'model__colsample_bytree': [0.6, 0.8]
    }
]

Os modelos de classificação escolhidos foram: 
- DummyClassifier: Modelo baseline, se um modelo mais complexo não performa significativamente melhor do que o DummyClassifier, isso pode indicar que o modelo precisa de mais desenvolvimento, ajuste ou que os dados não são adequados para o problema em questão já que ele não aprende com os dados. Também resolvi esse escolhe-lo pois o conjunto de dados é altamente desbalanceado (uma classe é muito mais frequente que outra) e o Dummy pode ser útil para mostrar como um modelo que sempre prevê a classe mais frequente se comportaria. Sempre bom validar porque modelos mais complexos podem parecer ter alta acurácia simplesmente prevendo sempre a classe dominante. Também servirá para uma análise estatística, pois posso comparar se um modelo tem um desempenho significativamente melhor do que um modelo que não tem capacidade de aprendizado.
- LogisticRegression
- XGBoost

Os outros modelos comentados serveriam para validação também, porém por questões de tempo e poder computacional, estarei deixando de fora do pipeline para o GridSearch processar mais rapidamente (em alguns testes anteriores, vi que o XGBoost se sairia melhor, então por isso estou o escolhendo ao invés do RandomForest, se eu deixasse os dois, que foi uma das minhas tentativas, a otimização exaustiva não daria certo). Caso queira verificar como os outros modelos comportariam nesse case, apenas tire os comentários e rode esse notebook.

O que eu quero atingir com as predições do meu modelo?
- Por ser um case voltado a cancelamentos, o que me importa é que meu modelo acerte o máximo possível de clientes que tem grande probabilidade de cancelar. A reflexão para definir isso é a seguinte: é melhor que o modelo classifique que um cliente vai cancelar mesmo que ele não vá ou que ele classifique que não vai cancelar um cliente que vai cancelar? Com isso, defino que prefiro aumentar os falsos positivos do que os falsos negativos. 
- Para esses tipos de situação, temos dois tipos de métrica que valem ser observadas: precisão e o recall. O trade-off dessas duas métricas é uma consideração importante na escolha de um limiar adequado ao meu modelo. Um alto recall pode ser mais desejável para garantir que todos os casos positivos sejam identificados, por outro lado, uma alta precisão pode ser preferida para evitar os falsos positivos. Em passos posteriores, vou aprofundar nas análises dessa questão, mas acho bom frisar isso.

"First off, it might not be good to just go by recall alone. You can simply achieve a recall of 100% by classifying everything as the positive class. I usually suggest using AUC for selecting parameters, and then finding a threshold for the operating point (say a given precision level) that you are interested in." - Vi esse comentário em um fórum, e pesquisando afundo e interpretando pro contexto do problema, defini não utilizar Recall para o GridSearch, e abaixo eu explico, porém, não deixarei de analisar essa métrica extremamente importante na próxima etapa.

In [ ]:
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=StratifiedKFold().split(X_train, y_train), 
                           scoring='roc_auc', verbose=2, error_score="raise")
grid_search.fit(X_train, y_train)

Algumas observações importantes sobre a montagem do GridSearch para essa situação:

- StratifiedKFold foi escolhido para preservar a proporção das classes, permitindo uma avaliação mais precisa e robusta do desempenho do modelo, evitando o viés que pode ocorrer se uma dobra tiver uma distribuição de classes significativamente diferente das outras.
- ROC_AUC foi escolhido como métrica para medir a capacidade do modelo de distinguir entre as classes, e no contexto do GridSearch, a otimização dos hiperparâmetros do modelo está direcionada para maximizar sua capacidade de discriminação entre as classes, ao invés de apenas maximizar a precisão geral. Também foi decidido utilizar ROC_AUC ao invés de Recall pois os dados são desbalanceados, e isso poderia afetar no objetivo final, já que (por exemplo) se o modelo classificar tudo como positivo, não haveria nenhum falso negativo, e eu teria um Recall de 1, o que daria uma falsa interpretabilidade de um bom score, no contexto da otimização dos huperparâmetros. Posteriormente, estarei usando todas as métricas para avalidação, mas nesse caso, preferi o ROC_AUC por isso.

In [61]:
grid_search.best_estimator_

Pipeline(steps=[('kmeans_cluster',
                 KMeansCluster(columns_cluster=['Latitude', 'Longitude'],
                               model_path='../preprocessing/kmeans_model.pkl')),
                ('import_drop',
                 DropColumns(drop_columns=['City', 'Latitude', 'Longitude',
                                           'ID'])),
                ('service_transformer',
                 ConditionalServiceTransformer(encoder_type='label')),
                ('categorical_encoder',
                 CategoricalEncoder(encoder_type='l...
                               feature_types=None, gamma=0.01, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=3, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

In [62]:
grid_search.best_score_

0.8634806918269178

O melhor modelo foi o XGBoost com um score (ROC_AUC) de 0.86 - isso significa que, na maioria dos limiares de decisão, o modelo tem uma alta taxa de verdadeiros positivos enquanto mantém uma baixa taxa de falsos positivos (que é o nosso objetivo como dito anteriormente). O valor de 0.86 também pode ser interpretado como a probabilidade de que, dado um par aleatório de observações (uma de cada classe), o modelo classificará corretamente a observação da classe positiva com uma pontuação mais alta do que a observação da classe negativa. 

Um ponto que veremos na próxima etapa, mas adianto aqui, é que quanto mais longe de 0.5 melhor. Um score ROC_AUC de 0.5 é, basicamente, um chute aleatório. Significaria que o modelo não tem capacidade de distinguir entre a classe positiva e a negativa melhor do que simplesmente jogando uma moeda, por exemplo.

Para tratamento dos dados, seguiremos com os clusters, o LabelEncoder, a transformação logarítimica e o StandardScaler.

In [63]:
gs_results = pd.DataFrame(grid_search.cv_results_)

In [64]:
gs_results['param_model'] = gs_results['param_model'].astype(str)
best_score = gs_results.loc[gs_results.groupby('param_model')['mean_test_score'].idxmax()].copy()
best_score

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_categorical_encoder__encoder_type,param_kmeans_cluster__active,param_model,param_rbf__active,param_scaler,param_service_transformer__encoder_type,...,param_model__subsample,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.391582,0.056593,0.132938,0.021243,onehot,True,DummyClassifier(strategy='most_frequent'),True,StandardScaler(),onehot,...,NaN,{'categorical_encoder__encoder_type': 'onehot'...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.000000,2881
194,0.163052,0.016638,0.105448,0.012171,onehot,True,LogisticRegression(),True,StandardScaler(),label,...,NaN,{'categorical_encoder__encoder_type': 'onehot'...,0.858750,0.862933,0.857959,0.854166,0.871080,0.860978,0.005768,267
1958,0.181935,0.037978,0.082483,0.025461,label,True,"XGBClassifier(base_score=None, booster='gbtree...",False,StandardScaler(),label,...,0.8,"{'categorical_encoder__encoder_type': 'label',...",0.859884,0.866093,0.861707,0.858520,0.871199,0.863481,0.004628,1


In [65]:
grid_search.best_params_

{'categorical_encoder__encoder_type': 'label',
 'kmeans_cluster__active': True,
 'model': XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=0.6, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=0.01, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=0.1, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=3, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=None, n_jobs=None,
               num_parallel_tree=None, random_state=None, ...),
 'model__booster': 'gbtree',
 'model__colsample_bytree': 0.6,
 'model__gamma': 0.01,
 'model__learning_rate': 0.1,
 'model__max_depth': 3,
 'm

In [69]:
modelpath = Path('../model')
cvspath = Path('../data/d04_models')

In [70]:
with open(f'{modelpath}/xgboost_params.txt', 'w') as file:
    file.write(str(grid_search.best_params_))

In [71]:
best_score.to_csv(f'{cvspath}/score_models.csv', index=False)